In [8]:
import scraper
from importlib import reload

reload(scraper)
from scraper import *

In [9]:
driver = initialize_otodom_scraper()

In [10]:
first_page_url = 'https://www.otodom.pl/pl/wyniki/wynajem/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?distanceRadius=0&limit=36&extras=%5BIS_PRIVATE_OWNER%5D&by=DEFAULT&direction=DESC&viewType=listing&page=2'
announcements_links = get_announcements_links(driver, first_page_url)

In [12]:
len(announcements_links)

2132